# Interpolation speed test

## User Specifications

In [1]:
### User can change these variables (unless they begin with an underscore) ###

drift_model = 'turnbull'  # must be 'turnbull' or 'wagner'
ocean_model = 'ecmwf'  # must be 'ecmwf'
atm_model = 'ecmwf'  # must be 'ecmwf'


iip_season = 2015  # must be 2015 (more to come later)

# Uncomment only one of the following
_method = 'index'; iip_indices = range(3283, 3285)  # iip_indices must be int or range
#_method = 'ID'; iip_ID = 20668  # iip_ID must be int

## Imports

In [12]:
from datetime import timedelta
import numpy as np

In [3]:
if drift_model == 'turnbull':
    from icedef.turnbull import drift
elif drift_model == 'wagner':
    from iceberg.wagner import drift
else:
    print('drift model specified is invalid')
    
from icedef import iceberg, metocean, plot

## Iceberg Data

In [4]:
if _method == 'index':
    identifier = iip_indices
elif _method == 'ID':
    identifier = iip_ID
else:
    print("Invalid method specified. Please change _method to either 'index' or 'ID'")

In [5]:
iip_berg_df, iip_berg = iceberg.get_iip_iceberg(2015, _method, identifier)

/home/evankielley/IceDEF/icedef/iceberg.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  iip_df['TIMESTAMP'] += pd.to_timedelta(pd.to_datetime(iip_df['SIGHTING_TIME'], format='%H%M').dt.hour, unit='h')
/home/evankielley/IceDEF/icedef/iceberg.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  iip_df['TIMESTAMP'] += pd.to_timedelta(pd.to_datetime(iip_df['SIGHTING_TIME'], format='%H%M').dt.minute, unit='m')


In [6]:
iip_berg_df

,index,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP
0,3283,2015,20668,6/16/2015,1440,48.20,-50.18,R/V,LG,TAB,GTJZ,2015-06-16 14:40:00
1,3284,2015,20668,6/17/2015,1043,48.16,-50.16,R/V,LG,GEN,GMRS,2015-06-17 10:43:00


In [7]:
mod_berg = iceberg.clone_iceberg_state(iip_berg)

## Metocean Data

In [8]:
%%time
ocean_data = metocean.ECMWF_Ocean(min(iip_berg.history['X']), max(iip_berg.history['X']),
                         min(iip_berg.history['Y']), max(iip_berg.history['Y']),
                         iip_berg.history['T'][0], iip_berg.history['T'][-1])

ocean_data.filenames

CPU times: user 1.4 s, sys: 2.15 s, total: 3.55 s
Wall time: 24.1 s


In [9]:
%%time
atm_data = metocean.ECMWF_Atm(min(iip_berg.history['X']), max(iip_berg.history['X']),
                         min(iip_berg.history['Y']), max(iip_berg.history['Y']),
                         iip_berg.history['T'][0], iip_berg.history['T'][-1])

atm_data.filenames

CPU times: user 42.9 ms, sys: 69.6 ms, total: 113 ms
Wall time: 2.04 s


## Timesteps

In [13]:
t0 = iip_berg.history['T'][0]
tf = iip_berg.history['T'][-1]
t_delta = tf - t0
t_delta_hours = t_delta.days*24 + t_delta.seconds/3600
t_step_hours = 0.1
t_step = timedelta(hours = t_step_hours)
dt = t_step_hours*3600  # timestep in seconds
t_all = []

for i in np.arange(0, t_delta_hours + t_step_hours, t_step_hours):
    new_date = t0 + timedelta(hours=i)
    t_all.append(new_date)

In [14]:
# Test Case
t0 = 573791.3994416415
y0 = 48.199906857329346
x0 = -50.172364036285984

In [ ]:
print(ocean_data.get_interpolated_velocities(t0, y0, x0))
%timeit ocean_data.get_interpolated_velocities(t0, y0, x0)

(0.16930037167741474, 0.04215040648384053)
2.98 µs ± 8.19 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
print(ocean_data.iUW([t0, y0, x0])[0], ocean_data.iVW([t0, y0, x0])[0])
%timeit ocean_data.iUW([t0, y0, x0])[0], ocean_data.iVW([t0, y0, x0])[0]

0.191034619659 0.00396931461185


In [16]:
data = ocean_data.VW

xmin, dx = ocean_data.xmin, ocean_data.dx
ymin, dy = ocean_data.ymin, ocean_data.dy
tmin, dt = ocean_data.tmin, ocean_data.dt

i = (x0 - xmin)/dx
j = (y0 - ymin)/dy
n = (t0 - tmin)/dt

(i0, di), (j0, dj), (n0, dn) = [(int(np.floor(x)), x - np.floor(x)) for x in [i, j, n]]

data[n0:n0+2, j0:j0+2, i0:i0+2]

array([[[ 0.01586962,  0.0122074 ],
        [ 0.06530961,  0.03295999]],

       [[-0.0061037 , -0.01037629],
        [ 0.04638813,  0.01831111]]], dtype=float32)